In [ ]:
!pip3 install pyrealsense2 open3d

In [7]:
!pip3 install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 9.0 MB/s eta 0:00:00m eta 0:00:01m0:00:01


In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
import open3d as o3d

# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
 
# Enable depth and color streams
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

# Get depth scale for converting depth values to meters
profile = pipeline.get_active_profile()
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()

# Intrinsic parameters
intrinsics = profile.get_stream(rs.stream.depth).as_video_stream_profile().get_intrinsics()
fx, fy, cx, cy = intrinsics.fx, intrinsics.fy, intrinsics.ppx, intrinsics.ppy

def depth_to_3d(x, y, depth):
    """ Convert pixel (x, y) and depth to 3D world coordinates. """
    z = depth * depth_scale
    if z == 0:  # Ignore invalid depth points
        return None
    X = (x - cx) * z / fx
    Y = (y - cy) * z / fy
    return np.array([X, Y, z])

def detect_ball(color_image):
    """ Detect a ball in the color image using color thresholding. """
    hsv = cv2.cvtColor(color_image, cv2.COLOR_BGR2HSV)
    
    # Define color range for the ball (adjust as needed)
    lower_red = np.array([0, 140, 180])
    upper_red = np.array([20, 230, 255])
    
    mask = cv2.inRange(hsv, lower_red, upper_red)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        (x, y), radius = cv2.minEnclosingCircle(largest_contour)
        if radius > 5:  # Filter out small noise
            return int(x), int(y), int(radius)
    return None

try:
    while True:
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        # Convert frames to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Detect the ball
        ball = detect_ball(color_image)
        if ball:
            x, y, radius = ball

            # Get depth value at the detected position
            depth_value = depth_image[y, x]
            ball_position = depth_to_3d(x, y, depth_value)

            if ball_position is not None:
                print(f"Ball detected at: {ball_position}")

                # Draw the detected ball on the color image
                cv2.circle(color_image, (x, y), radius, (0, 255, 0), 2)
                cv2.putText(color_image, f"3D: {ball_position[0]:.2f}, {ball_position[1]:.2f}, {ball_position[2]:.2f}", 
                            (x + 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Show images
        cv2.imshow("Color Image", color_image)
        # cv2.imshow("Depth Image", cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET))

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()


In [ ]:
import pyrealsense2 as rs
import cv2
import numpy as np

# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

def get_hsv_value(event, x, y, flags, param):
    """Callback function to get HSV values on mouse click."""
    if event == cv2.EVENT_LBUTTONDOWN:  # Left mouse click
        hsv_pixel = hsv[y, x]  # Get HSV value at clicked position
        print(f"HSV at ({x}, {y}): {hsv_pixel}")

try:
    while True:
        # Wait for a new frame
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue
        
        # Convert frame to numpy array
        color_image = np.asanyarray(color_frame.get_data())
        hsv = cv2.cvtColor(color_image, cv2.COLOR_BGR2HSV)  # Convert to HSV
        
        # Display image
        cv2.imshow("RealSense Camera", color_image)
        cv2.setMouseCallback("RealSense Camera", get_hsv_value)  # Set mouse click event

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    # Stop the RealSense pipeline
    pipeline.stop()
    cv2.destroyAllWindows()


In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2

# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()

# Enable depth and color streams
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

# Get depth scale for converting depth values to meters
profile = pipeline.get_active_profile()
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()

# Get camera intrinsics
intrinsics = profile.get_stream(rs.stream.depth).as_video_stream_profile().get_intrinsics()
fx, fy, cx, cy = intrinsics.fx, intrinsics.fy, intrinsics.ppx, intrinsics.ppy

# Function to convert depth to 3D world coordinates
def depth_to_3d(x, y, depth):
    z = depth * depth_scale
    if z == 0:  # Ignore invalid depth points
        return None
    X = (x - cx) * z / fx
    Y = (y - cy) * z / fy
    return np.array([X, Y, z])

# Initialize previous frame for motion detection
previous_gray = None

try:
    while True:
        # Capture frames
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        # Convert to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Convert to grayscale
        gray = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)

        # If first frame, store and continue
        if previous_gray is None:
            previous_gray = gray
            continue

        # Compute absolute difference between current and previous frame
        frame_diff = cv2.absdiff(previous_gray, gray)

        # Apply threshold to highlight differences
        _, thresh = cv2.threshold(frame_diff, 30, 255, cv2.THRESH_BINARY)

        # Find contours of the moving regions
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            (x, y), radius = cv2.minEnclosingCircle(largest_contour)
            
            if radius > 10:  # Filter out small noise
                x, y, radius = int(x), int(y), int(radius)

                # Get depth value at detected position
                depth_value = depth_image[y, x]
                ball_position = depth_to_3d(x, y, depth_value)

                if ball_position is not None:
                    print(f"Moving object detected at: {ball_position}")

                    # Draw detection circle
                    cv2.circle(color_image, (x, y), radius, (0, 255, 0), 2)
                    cv2.putText(color_image, f"3D: {ball_position[0]:.2f}, {ball_position[1]:.2f}, {ball_position[2]:.2f}", 
                                (x + 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Update previous frame
        previous_gray = gray.copy()

        # Display results
        cv2.imshow("Color Image", color_image)
        cv2.imshow("Motion Mask", thresh)

        # Exit on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()


Moving object detected at: [ 0.02195311 -0.05009727  3.18400015]
